In [1]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
from energy_cycle import energy_generating_cycle_test

# read model for E. coli K12
k12 = read_sbml_model('../GEMs/iML1515.xml')

# MAM2 module
RA mod

In [2]:
MAM2 = k12.copy()

## Metabolites

In [3]:
#L-arogenate
Largn_c = Metabolite(
    'Largn_c',
    formula='C10H13NO5',
    name='L-Arogenate',
    compartment='c')

#caffeic acid
caffa_c = Metabolite(
    'caffa_c',
    formula='C9H8O4',
    name='Caffeic acid',
    compartment='c')

#external caffeic acid
caffa_e = Metabolite(
    'caffa_e',
    formula='C9H8O4',
    name='Caffeic acid',
    compartment='e')

#caffeoyl CoA
caffcoa_c = Metabolite(
    'caffcoa_c',
    formula='C30H38N7O19P3S',
    name='Caffeoyl-CoA',
    compartment='c')

#3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_c = Metabolite(
    'saa_c',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='c')

#external 3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_e = Metabolite(
    'saa_e',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='e')

#rosmarinic acid
rosma_c = Metabolite(
    'rosma_c',
    formula='C18H16O8',
    name='Rosmarinic acid',
    compartment='c')

#external rosmarinic acid
rosma_e = Metabolite(
    'rosma_e',
    formula='C18H16O8',
    name='Rosmarinic acid',
    compartment='e')

#Oxaloacetate
oaa_c = k12.metabolites.get_by_id("oaa_c")

#Aspartate
asp = k12.metabolites.get_by_id("asp__L_c")

#prephenate
pphn_c = k12.metabolites.get_by_id("pphn_c")

#ATP
atp_c = k12.metabolites.get_by_id("atp_c")

#AMP
amp_c = k12.metabolites.get_by_id("amp_c")

#Diphosphate
ppi_c = k12.metabolites.get_by_id("ppi_c")

#CoA
coa_c = k12.metabolites.get_by_id("coa_c")

## Reactions

In [4]:
#prep -> l-arogenate
PPNARO = Reaction('PPNARO')
PPNARO.name = 'L-arogenate:2-oxoglutarate aminotransferase'
PPNARO.subsystem = 'Phenylalanine, tyrosine and tryptophan biosynthesis'
PPNARO.lower_bound = -1000  # This is the default
PPNARO.upper_bound = 1000  # This is the default
#Oxaloacetate + L-Arogenate <=> L-Aspartate + Prephenate
PPNARO.add_metabolites({
    oaa_c: -1.0,
    Largn_c: -1.0,
    asp: 1.0,
    pphn_c: 1.0
})
PPNARO.gene_reaction_rule = 'b4054' #tyrB

MAM2.add_reactions([PPNARO])

In [5]:
#caffeic acid transport reaction
CAFFAt = Reaction('CAFFAt')
CAFFAt.name = 'caffa transport'
CAFFAt.subsystem = 'RA module'
CAFFAt.lower_bound = -1000  # This is the default
CAFFAt.upper_bound = 1000  # This is the default
CAFFAt.add_metabolites({
    caffa_c: -1.0,
    caffa_e: 1.0
})

MAM2.add_reactions([CAFFAt])

In [6]:
#caffeic acid -> caffeoyl CoA
CAFFCOA = Reaction('CAFFCOA')
CAFFCOA.name = '4-coumarate:CoA ligase'
CAFFCOA.subsystem = 'RA module'
CAFFCOA.lower_bound = -1000  # This is the default
CAFFCOA.upper_bound = 1000  # This is the default
#ATP + Caffeate + CoA <=> AMP + Diphosphate + Caffeoyl-CoA
CAFFCOA.add_metabolites({
    atp_c: -1.0,
    caffa_c: -1.0,
    coa_c: -1.0,
    amp_c: 1.0,
    caffcoa_c: 1.0,
    ppi_c: 1.0
})
#CAFFCOA.gene_reaction_rule = '(cl4)' #synthetic 4CL gene

MAM2.add_reactions([CAFFCOA])

In [7]:
#SAA transport reaction
SAAt = Reaction('SAAt')
SAAt.name = 'caffa transport'
SAAt.subsystem = 'RA module'
SAAt.lower_bound = -1000  # This is the default
SAAt.upper_bound = 1000  # This is the default
SAAt.add_metabolites({
    saa_c: -1.0,
    saa_e: 1.0
})

MAM2.add_reactions([SAAt])

In [8]:
#salvianic acid A + caffeoyl CoA -> Rosmarinic acid
RAS = Reaction('RAS')
RAS.name = 'rosmarinic acid synthase'
RAS.subsystem = 'RA module'
RAS.lower_bound = -1000  # This is the default
RAS.upper_bound = 1000  # This is the default
#Caffeoyl-CoA + 3-(3,4-Dihydroxyphenyl)lactate <=> CoA + Rosmarinate
RAS.add_metabolites({
    caffcoa_c: -1.0,
    saa_c: -1.0,
    coa_c: 1.0,
    rosma_c: 1.0
})
#RAS.gene_reaction_rule = '(ras)' #synthetic 4CL gene

MAM2.add_reactions([RAS])

In [9]:
#RA transport reaction
RAt = Reaction('RAt')
RAt.name = 'caffa transport'
RAt.subsystem = 'RA module'
RAt.lower_bound = -1000  # This is the default
RAt.upper_bound = 1000  # This is the default
RAt.add_metabolites({
    rosma_c: -1.0,
    rosma_e: 1.0
})

MAM2.add_reactions([RAt])

In [10]:
#exhange reactions for SA, caffa, and RA
MAM2.add_boundary(MAM2.metabolites.get_by_id("caffa_e"), type="exchange")
MAM2.add_boundary(MAM2.metabolites.get_by_id("saa_e"), type="exchange")
MAM2.add_boundary(MAM2.metabolites.get_by_id("rosma_e"), type="exchange")

Reaction identifier,EX_rosma_e
Name,Rosmarinic acid exchange
Memory address,0x121c880a0
Stoichiometry,rosma_e <=> Rosmarinic acid <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Testing

In [11]:
energy_generating_cycle_test(MAM2)

ATPM flux: 0.0


,fluxes,reduced_costs
CYTDK2,0.0,-5.551115e-16
XPPT,0.0,1.015524e+01
HXPRT,0.0,1.015524e+01
NDPK5,0.0,-1.922381e+00
SHK3Dr,0.0,-2.000000e+00
...,...,...
RAS,0.0,-2.000000e+00
RAt,0.0,2.000000e+00
EX_caffa_e,0.0,-2.000000e+00
EX_saa_e,0.0,-2.000000e+00


In [12]:
#so that cell cannot eat rosmarinic acid
MAM2.reactions.EX_rosma_e.lower_bound = 0

In [13]:
from cobra.flux_analysis import flux_variability_analysis

MAM2.metabolites.rosma_c.summary(fva=0.95)

Percent,Flux,Range,Reaction,Definition
,0,[0; 6.661],RAS,caffcoa_c + saa_c <=> coa_c + rosma_c
Percent,Flux,Range,Reaction,Definition
,0,[-6.661; 0],RAt,rosma_c <=> rosma_e
